In [1]:
import numpy as np
import tensorflow as tf
from load_dataset_AlexNet import load_dataset
import matplotlib.pyplot as plt
import time

In [2]:
x_train,t_train = load_dataset('./train_dataset_AlexNet',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)
x_test,t_test = load_dataset('./test_dataset_AlexNet',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)

Load label : Done!
Load img : Done!
Load label : Done!
Load img : Done!


In [3]:
X = tf.placeholder(tf.float32, shape=[None, 224*224*3],name="input")
t = tf.placeholder(tf.float32, shape=[None, 4])
keep_prob = tf.placeholder(tf.float32,name="keep_prob")
### 入力層
input_layer = tf.reshape(X, [-1,224,224,3])

In [4]:
### 畳込み層、プーリング層1
h_conv1 = tf.layers.conv2d(inputs=input_layer,filters=96,kernel_size=[11, 11],strides=(4,4),padding='same',activation=tf.nn.relu)
h_norm1 = tf.nn.local_response_normalization(h_conv1,depth_radius=2,alpha=2e-5,beta=0.75)
h_pool1 = tf.layers.max_pooling2d(inputs=h_norm1,pool_size=(3,3),strides=(2,2),padding='same')

In [5]:
### 畳込み層、プーリング層2
h_conv2 = tf.layers.conv2d(inputs=h_pool1,filters=256,kernel_size=[5, 5],padding='same',activation=tf.nn.relu)
h_norm2 = tf.nn.local_response_normalization(h_conv2,depth_radius=2,alpha=2e-5,beta=0.75)
h_pool2 = tf.layers.max_pooling2d(inputs=h_norm2,pool_size=(3,3),strides=(2,2),padding='same')
print(h_norm2)
print(h_pool2)

Tensor("LRN_1:0", shape=(?, 28, 28, 256), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 14, 14, 256), dtype=float32)


In [6]:
### 畳込み層、プーリング層3
h_conv3_1 = tf.layers.conv2d(inputs=h_pool2,filters=384,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
h_conv3_2 = tf.layers.conv2d(inputs=h_conv3_1,filters=384,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
h_conv3_3 = tf.layers.conv2d(inputs=h_conv3_2,filters=256,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)

h_pool3_1 = tf.layers.max_pooling2d(inputs=h_conv3_3,pool_size=(14,14),strides=(2,2),padding='valid')
h_pool3_2 = tf.layers.max_pooling2d(inputs=h_conv3_3,pool_size=(12,12),strides=(2,2),padding='valid')
h_pool3_3 = tf.layers.max_pooling2d(inputs=h_conv3_3,pool_size=(9,9),strides=(2,2),padding='valid')
h_pool3_4 = tf.layers.max_pooling2d(inputs=h_conv3_3,pool_size=(4,4),strides=(2,2),padding='valid')
h_pool3_1_flat = tf.reshape(h_pool3_1,[-1,1*1*256])
h_pool3_2_flat = tf.reshape(h_pool3_2,[-1,2*2*256])
h_pool3_3_flat = tf.reshape(h_pool3_3,[-1,3*3*256])
h_pool3_4_flat = tf.reshape(h_pool3_4,[-1,6*6*256])

print(h_conv3_3)
print(h_pool3_1_flat)
print(h_pool3_2_flat)
print(h_pool3_3_flat)
print(h_pool3_4_flat)

h_spp = tf.concat([h_pool3_1_flat, h_pool3_2_flat, h_pool3_3_flat, h_pool3_4_flat],axis=1)
print(h_spp)

Tensor("conv2d_5/Relu:0", shape=(?, 14, 14, 256), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 256), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 1024), dtype=float32)
Tensor("Reshape_3:0", shape=(?, 2304), dtype=float32)
Tensor("Reshape_4:0", shape=(?, 9216), dtype=float32)
Tensor("concat:0", shape=(?, 12800), dtype=float32)


In [7]:
### 全結合層1

stddev = np.sqrt(1.0 / 12800)
h_W_fc1 = tf.Variable(tf.truncated_normal([12800,4096], stddev=stddev)) 
h_b_fc1 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_fc1 = tf.nn.relu(tf.matmul(h_spp, h_W_fc1) + h_b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [8]:
### 全結合層2

stddev = np.sqrt(1.0 / 4096)
h_W_fc2 = tf.Variable(tf.truncated_normal([4096,4096], stddev=stddev)) 
h_b_fc2 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, h_W_fc2) + h_b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [9]:
### 出力層
stddev = np.sqrt(2.0 / 4096)
W_fc3 = tf.Variable(tf.truncated_normal([4096,4], stddev=stddev))
b_fc3 = tf.Variable(tf.constant(0.1, shape=[4]))
y_conv = tf.nn.xw_plus_b(h_fc2_drop,W_fc3,b_fc3,name="output")

In [10]:
### 損失
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y_conv))

### 学習op
optimizer = tf.train.AdamOptimizer(1e-5)
train_step = optimizer.minimize(loss)

### モデルの評価
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(t,1))
# 精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 30
start = time.time()
for i in range(300):
    batch_mask = np.random.choice(x_train.shape[0], batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: t_batch,keep_prob:0.5})
    if i % 10 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: x_train, t: t_train,keep_prob:1.0})
        test_acc = sess.run(accuracy, feed_dict={X: x_test, t: t_test,keep_prob:1.0})
        print("[Train] step: %d, loss: %f, acc: %f, [Test] acc : %f" % (i, train_loss, train_acc,test_acc))
        print("Time : %f" % (time.time() - start))
        
saver.save(sess, "./RoadSign-Alexnet-spp-ckpt/roadsign_alex_spp")

[Train] step: 0, loss: 1.354834, acc: 0.370900, [Test] acc : 0.405405
Time : 35.224357
[Train] step: 10, loss: 1.342921, acc: 0.273339, [Test] acc : 0.162162
Time : 89.294784
[Train] step: 20, loss: 1.315431, acc: 0.370900, [Test] acc : 0.405405
Time : 152.735984
[Train] step: 30, loss: 1.296959, acc: 0.370900, [Test] acc : 0.405405
Time : 211.458777
[Train] step: 40, loss: 1.277008, acc: 0.370900, [Test] acc : 0.405405
Time : 268.845662
[Train] step: 50, loss: 1.231085, acc: 0.403701, [Test] acc : 0.405405
Time : 323.676125
[Train] step: 60, loss: 1.151875, acc: 0.506308, [Test] acc : 0.648649
Time : 376.132766
[Train] step: 70, loss: 1.058659, acc: 0.574432, [Test] acc : 0.648649
Time : 441.771985
[Train] step: 80, loss: 0.984500, acc: 0.513036, [Test] acc : 0.675676
Time : 504.283124
[Train] step: 90, loss: 0.932609, acc: 0.518923, [Test] acc : 0.648649
Time : 564.317171
[Train] step: 100, loss: 0.955021, acc: 0.532380, [Test] acc : 0.540541
Time : 621.066628
[Train] step: 110, loss

'./RoadSign-Alexnet-spp-ckpt/roadsign_alex_spp'

In [13]:
ckpt = tf.train.get_checkpoint_state('./RoadSign-Alexnet-spp-ckpt')
if ckpt:
    # checkpointファイルから最後に保存したモデルへのパスを取得する
    last_model = ckpt.model_checkpoint_path
    print("load {0}".format(last_model))
    # 学習済みモデルを読み込む
    saver.restore(sess, last_model)

batch_size = 30
start = time.time()
for i in range(500):
    batch_mask = np.random.choice(x_train.shape[0], batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: t_batch,keep_prob:0.5})
    if i % 20 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: x_train, t: t_train,keep_prob:1.0})
        test_acc = sess.run(accuracy, feed_dict={X: x_test, t: t_test,keep_prob:1.0})
        print("[Train] step: %d, loss: %f, acc: %f, [Test] acc : %f" % (i, train_loss, train_acc,test_acc))
        print("Time : %f" % (time.time() - start))
        
saver.save(sess, "./RoadSign-Alexnet-spp-ckpt2/roadsign_alex_spp")

load ./RoadSign-Alexnet-spp-ckpt/roadsign_alex_spp
INFO:tensorflow:Restoring parameters from ./RoadSign-Alexnet-spp-ckpt/roadsign_alex_spp
[Train] step: 0, loss: 0.551279, acc: 0.757780, [Test] acc : 0.837838
Time : 35.775541
[Train] step: 20, loss: 0.516316, acc: 0.777124, [Test] acc : 0.837838
Time : 129.885311
[Train] step: 40, loss: 0.497363, acc: 0.783852, [Test] acc : 0.729730
Time : 216.746541
[Train] step: 60, loss: 0.489088, acc: 0.787216, [Test] acc : 0.729730
Time : 306.049734
[Train] step: 80, loss: 0.477144, acc: 0.804037, [Test] acc : 0.675676
Time : 396.907464
[Train] step: 100, loss: 0.418703, acc: 0.816653, [Test] acc : 0.729730
Time : 485.095378
[Train] step: 120, loss: 0.498485, acc: 0.793944, [Test] acc : 0.621622
Time : 571.354650
[Train] step: 140, loss: 0.456117, acc: 0.788898, [Test] acc : 0.837838
Time : 656.997699
[Train] step: 160, loss: 0.355467, acc: 0.847771, [Test] acc : 0.810811
Time : 737.250022
[Train] step: 180, loss: 0.441363, acc: 0.804037, [Test] a

'./RoadSign-Alexnet-spp-ckpt2/roadsign_alex_spp'